In [1]:
import pandas as pd
import numpy as np

In [17]:
#from census import Census
#from us import states
#import censusdata

#c = Census("181f11c579e665cf03575f5cc8313e0d612020b8", year=2019)

In [2]:
starbuck_df = pd.read_csv('starbucks_directory.csv')
acs_df = pd.read_csv('acs_df.csv')
county_df = pd.read_csv('ZIP-COUNTY-FIPS_2017-06.csv')

In [52]:
# Filter for only US locations
starbuck_df = starbuck_df[starbuck_df.Country == 'US']

# Make zip to 5 digits
starbuck_df['Postcode'] = starbuck_df['Postcode'].str[:5]

# Display stores w/o zip
starbuck_df[starbuck_df['Postcode'].isnull()] #Address 4546 Boardman Canfield Road, Canfield, OH 44406

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
21549,Starbucks,28180-250430,Canfield,Company Owned,4546 Boardman Canfield Road,Canfield,OH,US,NaN,NaN,GMT-05:00 America/New_York,-80.74,41.03


In [53]:
# Replace null value for zip
starbuck_df.loc[21549, 'Postcode'] = 44406

In [54]:
# Change zip 
starbuck_df['Postcode'] = starbuck_df['Postcode'].astype(int)

In [39]:
starbuck_df = starbuck_df[starbuck_df.Country == 'US']
starbuck_df['Postcode'] = starbuck_df['Postcode'].str[:5]
starbuck_df['Postcode'] = starbuck_df['Postcode'].astype(int)

ValueError: cannot convert float NaN to integer

In [32]:
starbucks = starbuck_df.value_counts(subset=['Postcode']).reset_index(name='starbucks_stores')
starbucks = pd.DataFrame(starbucks)

starbucks

,Postcode,starbucks_stores
0,89109,32
1,98101,24
2,90045,21
3,92101,20
4,10019,20
...,...,...
6001,54449,1
6002,54467,1
6003,54481,1
6004,54650,1


In [34]:
#Merge county_df to acs_df
acs_df = acs_df.merge(county_df, left_on='zip_code', right_on='ZIP', how='left')
acs_df.columns = acs_df.columns.str.lower()
acs_df = acs_df.drop(['zip','stcountyfp','classfp'], axis=1)

In [35]:
acs = acs_df.iloc[:, np.r_[0:4, 44:52]]

In [36]:
acs['population_18_40'] = acs_df.iloc[:, np.r_[4:11, 23:30]].sum(axis=1)
acs['workers'] = acs_df.iloc[:, 42:44].sum(axis=1)

<ipython-input-36-ca8f5069fbe4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acs['population_18_40'] = acs_df.iloc[:, np.r_[4:11, 23:30]].sum(axis=1)
<ipython-input-36-ca8f5069fbe4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acs['workers'] = acs_df.iloc[:, 42:44].sum(axis=1)


In [38]:
acs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 462298 entries, 0 to 462297
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   zip_code              462298 non-null  int64  
 1   total_populatione     462298 non-null  int64  
 2   median_agee           461329 non-null  float64
 3   median_hh_incomee     455839 non-null  float64
 4   median_home_valuee    455223 non-null  float64
 5   walk_to_worke         462298 non-null  int64  
 6   total_householdse     462298 non-null  int64  
 7   high_school_diplomae  462298 non-null  int64  
 8   bachelors_degreee     462298 non-null  int64  
 9   masters_degreee       462298 non-null  int64  
 10  countyname            462189 non-null  object 
 11  state                 462189 non-null  object 
 12  population_18_40      462298 non-null  int64  
 13  workers               462298 non-null  int64  
dtypes: float64(3), int64(9), object(2)
memory usage: 52.

In [37]:
#Merge Starbucks stores count to acs
df = acs.merge(starbucks, left_on='zip_code', right_on='Postcode', how='left')
df = df.drop(['Postcode'], axis=1)

df.head()

ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat

In [105]:
starbucks_df.groupby(['revenue','session','user_id'])['user_id'].count()

,female_18_19e,female_20e,female_21e,female_22_24e,female_25_29e,female_30_34e,female_35_39e
0,0,20,0,0,0,20,40
1,0,0,0,26,22,35,0
2,0,0,0,11,21,54,42
3,0,0,0,11,21,54,42
4,11,15,29,64,337,216,262
...,...,...,...,...,...,...,...
46519,0,0,0,0,0,0,0
46520,0,0,0,0,0,0,28
46521,0,0,0,0,0,0,5
46522,2,0,0,0,0,5,2
